In [ ]:
import pandas as pd
import nltk
import re
import math
import numpy as np
from gensim.models import Word2Vec
import gensim
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn import datasets
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import numpy as np
from nltk.stem import WordNetLemmatizer
from keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

from keras.utils import to_categorical
from keras import backend as K

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.ensemble import StackingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
import xgboost
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/Hands on/Data/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Hands on/Data/test.csv")

In [ ]:
df_train = df_train.drop(["Unnamed: 0"],axis=1)
df_test = df_test.drop(["Unnamed: 0"],axis=1)

In [ ]:
df_train.head()

,text,label,cleaned_text
0,Wall St. Bears Claw Back Into the Black (Reute...,2,wall st bears claw back black reuters reuters ...
1,Carlyle Looks Toward Commercial Aerospace (Reu...,2,carlyle looks toward commercial aerospace reut...
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2,oil economy cloud stocks outlook reuters reute...
3,Iraq Halts Oil Exports from Main Southern Pipe...,2,iraq halts oil exports main southern pipeline ...
4,"Oil prices soar to all-time record, posing new...",2,oil prices soar alltime record posing new mena...


In [ ]:
len(df_train)

120000

In [ ]:
df_train = df_train.sample(frac=1).reset_index(drop=True)

In [ ]:
df_train.head()

,text,label,cleaned_text
0,PACs' Post-Labor Day Spending Favors Kerry (AP...,0,pacs postlabor day spending favors kerry ap ap...
1,Dollar Jumps as the G7 Fades LONDON (Reuters)...,2,dollar jumps g7 fades london reuters dollar su...
2,"Rein in militias, UN tells Sudan THE top UN en...",0,rein militias un tells sudan top un envoy suda...
3,Rooney wins Law praise Ex-Manchester United st...,1,rooney wins law praise exmanchester united str...
4,Viewpoint: School finance deadline unrealistic...,2,viewpoint school finance deadline unrealistic ...


In [ ]:
len(df_train)

120000

## Traditional ML Methods - HashingVector

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer

In [ ]:
vectorizer = HashingVectorizer(n_features=500)
X_train = vectorizer.fit_transform(df_train['cleaned_text'])
X_test = vectorizer.transform(df_test['cleaned_text'])
labels = df_train['label'].values
le = LabelEncoder()
le.fit(labels)
y_train = le.transform(labels)
y_test = le.transform(df_test['label'].values)
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

In [ ]:
X_train_dense.shape

(120000, 500)

In [ ]:
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.1, 1, 10],
    'solver': ['liblinear', 'lbfgs', 'saga']
}
grid_search = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5)
grid_search.fit(X_train_dense[:60000], y_train[:60000])
best_params = grid_search.best_params_

best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test_dense, y_test)
print("Train Accuracy: ", accuracy)
print("Test Accuracy: ",best_model.score(X_test_dense, y_test))
y_pred=best_model.predict(X_test_dense)
print(classification_report(y_test, y_pred, labels=[0,1,2,3]))
grid_search.best_estimator_

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
20 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solv

Train Accuracy:  0.729078947368421
Test Accuracy:  0.729078947368421
              precision    recall  f1-score   support

           0       0.74      0.75      0.74      1900
           1       0.76      0.80      0.78      1900
           2       0.71      0.69      0.70      1900
           3       0.71      0.68      0.69      1900

    accuracy                           0.73      7600
   macro avg       0.73      0.73      0.73      7600
weighted avg       0.73      0.73      0.73      7600



LogisticRegression(C=1, max_iter=1000, solver='liblinear')

In [ ]:
lr = LogisticRegression(C=1, max_iter=10000, penalty='l1', solver='saga')
lr.fit(X_train_dense, y_train)
print("Test Accuracy: ",lr.score(X_test_dense, y_test))
y_pred=lr.predict(X_test_dense)
print(classification_report(y_test, y_pred, labels=[0,1,2,3]))

Test Accuracy:  0.7276315789473684
              precision    recall  f1-score   support

           0       0.74      0.74      0.74      1900
           1       0.76      0.80      0.78      1900
           2       0.71      0.69      0.70      1900
           3       0.70      0.68      0.69      1900

    accuracy                           0.73      7600
   macro avg       0.73      0.73      0.73      7600
weighted avg       0.73      0.73      0.73      7600



In [ ]:
from sklearn.neural_network import MLPClassifier
param_grid = {
    'hidden_layer_sizes': [(256,), (128, 128), (128, 64, 32)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['adam'],
}

mlp = MLPClassifier(max_iter=1000)
grid_search = GridSearchCV(mlp, param_grid, cv=5)
grid_search.fit(X_train[:12000], y_train[:12000])
best_params = grid_search.best_params_

# Step 3: Model Evaluation
best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test, y_test)
print("Accuracy:", accuracy)
best_model

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy: 0.7405263157894737


MLPClassifier(hidden_layer_sizes=(256,), max_iter=1000)

In [ ]:
mlp = MLPClassifier(alpha=0.1,solver="adam",hidden_layer_sizes=(256,),max_iter=10000).fit(X_train,y_train)
print("Test Accuracy: ",mlp.score(X_test, y_test))
y_pred=mlp.predict(X_test)
print(classification_report(y_test, y_pred, labels=[0,1,2,3]))

Test Accuracy:  0.8025
              precision    recall  f1-score   support

           0       0.81      0.82      0.82      1900
           1       0.83      0.87      0.85      1900
           2       0.78      0.78      0.78      1900
           3       0.79      0.75      0.77      1900

    accuracy                           0.80      7600
   macro avg       0.80      0.80      0.80      7600
weighted avg       0.80      0.80      0.80      7600



In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler,MaxAbsScaler
scaler = MinMaxScaler()
scaler.fit(X_train_dense)
X_train2 = scaler.transform(X_train_dense)
scaler = MinMaxScaler()
scaler.fit(X_test_dense)
X_test2 = scaler.transform(X_test_dense)
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0]  # Example values for alpha
}
mnb = MultinomialNB()
grid_search = GridSearchCV(mnb, param_grid, cv=5)
grid_search.fit(X_train2, y_train)
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test2, y_test)
print("Accuracy:", accuracy)
best_model

Accuracy: 0.6385526315789474


MultinomialNB(alpha=0.1)

In [ ]:
print("Test Accuracy: ",best_model.score(X_test2, y_test))
y_pred=best_model.predict(X_test2)
print(classification_report(y_test, y_pred, labels=[0,1,2,3]))

Test Accuracy:  0.6385526315789474
              precision    recall  f1-score   support

           0       0.57      0.80      0.67      1900
           1       0.61      0.85      0.71      1900
           2       0.85      0.35      0.50      1900
           3       0.70      0.55      0.61      1900

    accuracy                           0.64      7600
   macro avg       0.68      0.64      0.62      7600
weighted avg       0.68      0.64      0.62      7600



In [ ]:

dt_classifier = DecisionTreeClassifier()
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [40],
    'min_samples_split': [2, 5, 10],
    'max_features': ['auto', 'sqrt', 'log2']
}

grid_search = GridSearchCV(dt_classifier, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_dense, y_train)
best_dt = grid_search.best_estimator_
y_pred = best_dt.predict(X_test)

print("Best Parameters:", grid_search.best_params_)
print("Best Estimator:", best_dt)
accuracy = best_dt.score(X_test_dense, y_test)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/pytho

Best Parameters: {'criterion': 'gini', 'max_depth': 40, 'max_features': 'sqrt', 'min_samples_split': 2}
Best Estimator: DecisionTreeClassifier(max_depth=40, max_features='sqrt')
Accuracy: 0.5796052631578947
Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.53      0.60      1900
           1       0.70      0.63      0.66      1900
           2       0.59      0.48      0.53      1900
           3       0.45      0.68      0.54      1900

    accuracy                           0.58      7600
   macro avg       0.60      0.58      0.58      7600
weighted avg       0.60      0.58      0.58      7600



In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion= 'gini', max_features='sqrt', min_samples_split= 2)
clf = clf.fit(X_train_dense, y_train)
accuracy = clf.score(X_test_dense, y_test)
print("Test Accuracy: ",clf.score(X_test_dense, y_test))
y_pred=clf.predict(X_test_dense)
print(classification_report(y_test, y_pred, labels=[0,1,2,3]))

Test Accuracy:  0.6218421052631579
              precision    recall  f1-score   support

           0       0.63      0.64      0.64      1900
           1       0.67      0.68      0.67      1900
           2       0.61      0.59      0.60      1900
           3       0.58      0.58      0.58      1900

    accuracy                           0.62      7600
   macro avg       0.62      0.62      0.62      7600
weighted avg       0.62      0.62      0.62      7600



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
rf_classifier = RandomForestClassifier(n_jobs=10,n_estimators=100)
rf_classifier.fit(X_train_dense, y_train)
print("Test Accuracy: ",rf_classifier.score(X_test_dense, y_test))
y_pred=rf_classifier.predict(X_test_dense)
print(classification_report(y_test, y_pred, labels=[0,1,2,3]))

Test Accuracy:  0.8007894736842105
              precision    recall  f1-score   support

           0       0.82      0.81      0.81      1900
           1       0.81      0.87      0.84      1900
           2       0.79      0.76      0.77      1900
           3       0.79      0.76      0.78      1900

    accuracy                           0.80      7600
   macro avg       0.80      0.80      0.80      7600
weighted avg       0.80      0.80      0.80      7600



In [ ]:
estimators = [
    ('mlp',MLPClassifier(alpha=0.1,solver="adam",hidden_layer_sizes=(256,),max_iter=10000)),
    ('mnb', MultinomialNB(alpha=0.1)),
    ('lr', LogisticRegression(C=10, max_iter=10000, penalty='l1', solver='saga')),
    ('dt', DecisionTreeClassifier(criterion= 'gini', max_features='sqrt', min_samples_split= 10))
]

estimators

[('mlp', MLPClassifier(alpha=0.1, hidden_layer_sizes=(256,), max_iter=10000)),
 ('mnb', MultinomialNB(alpha=0.1)),
 ('lr', LogisticRegression(C=10, max_iter=10000, penalty='l1', solver='saga')),
 ('dt',
  DecisionTreeClassifier(criterion='entropy', max_features='sqrt',
                         min_samples_split=10))]

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train_dense)
X_train2 = scaler.transform(X_train_dense)
scaler = MinMaxScaler()
scaler.fit(X_test_dense)
X_test2 = scaler.transform(X_test_dense)
model_voting = VotingClassifier(
    estimators = estimators,
    voting='hard',
    n_jobs=-1,
    flatten_transform=True,
    verbose=1).fit(X_train2, y_train)

print(model_voting.score(X_test2, y_test))
y_pred=model_voting.predict(X_test2)
print(classification_report(y_test, y_pred, labels=[0,1,2,3]))

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


0.6521052631578947
              precision    recall  f1-score   support

           0       0.57      0.82      0.67      1900
           1       0.62      0.87      0.72      1900
           2       0.83      0.43      0.56      1900
           3       0.78      0.49      0.60      1900

    accuracy                           0.65      7600
   macro avg       0.70      0.65      0.64      7600
weighted avg       0.70      0.65      0.64      7600



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf_gbc = GradientBoostingClassifier(n_estimators=50, learning_rate=1.0, random_state=0).fit(X_train_dense, y_train)
print(clf_gbc.score(X_test_dense, y_test))
y_pred=clf_gbc.predict(X_test_dense)
print(classification_report(y_test, y_pred, labels=[0,1,2,3]))

0.7823684210526316
              precision    recall  f1-score   support

           0       0.79      0.78      0.79      1900
           1       0.82      0.86      0.84      1900
           2       0.77      0.75      0.76      1900
           3       0.74      0.74      0.74      1900

    accuracy                           0.78      7600
   macro avg       0.78      0.78      0.78      7600
weighted avg       0.78      0.78      0.78      7600

